In [51]:
##Importing  Libraries

import pandas as pd
import numpy as np
import sklearn as sklearn
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier

In [52]:
df = pd.read_csv(r"C:\\Users\\hp\\Desktop\\HPA\\Anupam Bharteya\\data2.csv", nrows=25000)

df.shape

(25000, 46)

In [53]:
df_sample = df.sample(n=20, replace=False)

In [54]:
df = df.drop_duplicates()

In [55]:
df = df.dropna()

In [56]:
df = df.replace(to_replace = '?', value = "UNKNOWN")

In [57]:
df.shape

(25000, 46)

In [58]:
# Notice, there are quite a few missing ("UNKNOWNS") in the data  

# Let us explore .... 

# How many "UNKNOWN" entries are there in my data   

max_rows=df.shape[0]    # storing the 'max row number in data' as a value in object named 'max_rows' 
unknowns = []           # define a blank array in which we will store the num for each column 
colnames = df.columns      
var_counter=1 
for col in colnames:
    row_num=0 
    unknowns_count=0
    for row_num in range(0, max_rows):
        if (df[col].loc[row_num]=='UNKNOWN'):
            unknowns_count=unknowns_count+1
    #end loop2
    print('\n---------------- \n  ',var_counter, ' : ', col,'  has  ', unknowns_count, 'unknown values \n')
    unknowns = np.append(unknowns,unknowns_count)  # fill this value into the array  
    var_counter = var_counter + 1   
#end loop1

  


---------------- 
   1  :  encounter_id   has   0 unknown values 


---------------- 
   2  :  patient_nbr   has   0 unknown values 


---------------- 
   3  :  race   has   564 unknown values 


---------------- 
   4  :  gender   has   0 unknown values 


---------------- 
   5  :  age   has   0 unknown values 


---------------- 
   6  :  weight   has   24227 unknown values 


---------------- 
   7  :  admission_type_id   has   0 unknown values 


---------------- 
   8  :  discharge_disposition_id   has   0 unknown values 


---------------- 
   9  :  admission_source_id   has   0 unknown values 


---------------- 
   10  :  time_in_hospital   has   0 unknown values 


---------------- 
   11  :  payer_code   has   9954 unknown values 


---------------- 
   12  :  num_lab_procedures   has   0 unknown values 


---------------- 
   13  :  num_procedures   has   0 unknown values 


---------------- 
   14  :  num_medications   has   0 unknown values 


---------------- 
   15  :

In [59]:
# 'unknowns' is an array 
# colnames is a list 
# Both can be combined into a dataframe for better information visibility

In [60]:
df_unknowns = pd.DataFrame({"variable":colnames,"missing values":unknowns})


In [61]:
var_uniq_count = []       # define a blank array in which we will store the num for each column 
colnames = df.columns     # put all column names of the dataframe into a list 
counter=1
for col in colnames:
    x = len(df[col].unique())   # how many unique values does this variable take in the data 
    # print('\n','Data Column:',counter, '   ', col, ' - has ',x,'  unique values', '\n\n')
    var_uniq_count = np.append(var_uniq_count,x)  # fill this value into the array  
    counter=counter+1
    #for loop ends 
    
df_unique_values = pd.DataFrame({"variable" : colnames, "u_values": var_uniq_count})
df_unique_values

,variable,u_values
0,encounter_id,25000.0
1,patient_nbr,22087.0
2,race,6.0
3,gender,3.0
4,age,10.0
5,weight,8.0
6,admission_type_id,8.0
7,discharge_disposition_id,25.0
8,admission_source_id,14.0
9,time_in_hospital,14.0


In [62]:
def frequency_table (x,df):
    freq = df[x].value_counts()
    return freq

In [63]:
for col in colnames[2:]:
    print ("\n\n----------------") 
    print (col)
    print(frequency_table(col,df))
    



----------------
race
Caucasian          18565
AfricanAmerican     4863
UNKNOWN              564
Hispanic             486
Other                356
Asian                166
Name: race, dtype: int64


----------------
gender
Female             13488
Male               11511
Unknown/Invalid        1
Name: gender, dtype: int64


----------------
age
[70-80)     6442
[60-70)     5536
[80-90)     4237
[50-60)     4208
[40-50)     2336
[30-40)      926
[90-100)     720
[20-30)      406
[10-20)      163
[0-10)        26
Name: age, dtype: int64


----------------
weight
UNKNOWN      24227
[75-100)       312
[50-75)        231
[100-125)      162
[125-150)       29
[25-50)         27
[0-25)           9
[150-175)        3
Name: weight, dtype: int64


----------------
admission_type_id
1    13174
2     4620
3     4562
6     1319
5     1241
8       74
7        7
4        3
Name: admission_type_id, dtype: int64


----------------
discharge_disposition_id
1     14753
3      3446
6      3151
18      

In [64]:
print("original number of columns in data :  ", len(colnames),'\n')

# Create a filter so that we can identify which columns have only one value entered in them (across all rows)  
filt = (var_uniq_count < 2.0)

# Use this filter to select column names where there is just 1 unique value 
columns_with_one_uniq_value = colnames[filt]
print("number of columns with 1 unique value  :  ",  len(columns_with_one_uniq_value),'\n')

print("these columns are  : " , columns_with_one_uniq_value)
print('\n'*2)

print("In the next step, we will drop these from our data",'\n'*2)




original number of columns in data :   46 

number of columns with 1 unique value  :   6 

these columns are  :  Index(['acetohexamide', 'troglitazone', 'examide', 'citoglipton',
       'glimepiride.pioglitazone', 'metformin.pioglitazone'],
      dtype='object')



In the next step, we will drop these from our data 




In [65]:
print ("creating a new data frame df2 \n" 
"which does not contain \n the columns having 1 unique value \n")

df2 = df.drop(columns = columns_with_one_uniq_value)

# Or we can drop the columns in our original dataframe permanently, like this: 
# df.drop(columns = columns_with_one_uniq_value, inplace=True)

print("new data frame (rows/columns)  :", df2.shape,'\n'*2)


creating a new data frame df2 
which does not contain 
 the columns having 1 unique value 

new data frame (rows/columns)  : (25000, 40) 




In [66]:
var_uniq_count = []       # define a blank array in which we will store the num for each column 
colnames = df2.columns    # reading names of all columns into array names colnames 
counter=1
for col in colnames:
    x = len(df2[col].unique())   # how many unique values does this variable take in the data 
    print('\n','Data Column:',counter, '   ', col, ' - has ',x,'  unique values', '\n\n')
    var_uniq_count = np.append(var_uniq_count,x)  # fill this value into the array  
    counter=counter+1
    #for loop ends 


 Data Column: 1     encounter_id  - has  25000   unique values 



 Data Column: 2     patient_nbr  - has  22087   unique values 



 Data Column: 3     race  - has  6   unique values 



 Data Column: 4     gender  - has  3   unique values 



 Data Column: 5     age  - has  10   unique values 



 Data Column: 6     weight  - has  8   unique values 



 Data Column: 7     admission_type_id  - has  8   unique values 



 Data Column: 8     discharge_disposition_id  - has  25   unique values 



 Data Column: 9     admission_source_id  - has  14   unique values 



 Data Column: 10     time_in_hospital  - has  14   unique values 



 Data Column: 11     payer_code  - has  17   unique values 



 Data Column: 12     num_lab_procedures  - has  111   unique values 



 Data Column: 13     num_procedures  - has  7   unique values 



 Data Column: 14     num_medications  - has  72   unique values 



 Data Column: 15     number_outpatient  - has  25   unique values 



 Data Column: 16   

In [67]:


df3 = df2.drop(columns = ['weight', 'payer_code','tolbutamide', 'glipizide.metformin', 'metformin.rosiglitazone', 'miglitol', 'tolazamide'])



In [68]:
df3.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'insulin', 'glyburide.metformin', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object')

In [69]:
# Separate them out into two separate lists  -  NUMERIC and NOMINAL 

numeric_columns = ['time_in_hospital','num_lab_procedures', 'num_procedures', 'num_medications','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
nominal_columns = ['race', 'gender', 'age', 'admission_type_id','discharge_disposition_id',  'admission_source_id', 'max_glu_serum', 'A1Cresult','metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',  'rosiglitazone', 'acarbose', 'insulin', 'glyburide.metformin', 'change', 'diabetesMed']

# Also make a single list consisting of all x_variables 

x_variables = numeric_columns + nominal_columns

# Check 
len(x_variables)


30

In [70]:
# First we define a function so that we can iterate for all rows in the column 

def labelencoding (df,feature):
    le = preprocessing.LabelEncoder()      
    # just giving a name to the function  
    #This fn is the one that actually converts selected data to a unique numeric value 
    x = df[feature]
    le_obj = le.fit(x)
    feature = le_obj.transform(x)
    feature_val = np.reshape(feature,[len(feature),1])
    return feature_val


In [71]:
# Read all nominal value columns in an array & all numeric into another 
array_nominal = pd.DataFrame(df3,columns = nominal_columns).values
array_numeric = pd.DataFrame(df3,columns = numeric_columns).values


In [72]:
# Check
array_numeric.shape
array_nominal.shape 
array_nominal
array_numeric

array([[ 7, 62,  0, ...,  0,  0,  7],
       [ 4, 45,  4, ...,  0,  0,  8],
       [ 2, 32,  3, ...,  0,  0,  9],
       ...,
       [ 3, 30,  1, ...,  0,  1,  9],
       [ 3, 54,  1, ...,  0,  0,  6],
       [ 3, 13,  3, ...,  0,  1,  9]], dtype=int64)

In [73]:

# Now, replace all the values in the nominal value columns with unique numbers 
# and append each of those columns into array_numeric 
# so that we have one single collection of refined X-Variables to use in our model build

for col in nominal_columns:
    x = labelencoding(df3,col)
    array_numeric =  np.hstack((array_numeric,x))    
# Loop ends    
    
# now check the array 
array_numeric.shape
# and check a sample extract 
array_numeric[30:40,0:15]




array([[ 3, 36,  0, 15,  0,  0,  0,  6,  2,  0,  8,  0,  0,  6,  2],
       [ 6, 69,  0, 17,  0,  0,  3,  8,  2,  1,  6,  0,  2,  6,  2],
       [10, 67,  1, 24,  0,  0,  0,  8,  2,  0,  7,  0,  5,  6,  2],
       [ 4, 57,  0, 13,  0,  0,  1,  9,  2,  0,  9,  0,  0,  6,  2],
       [ 5, 43,  2, 16,  0,  0,  0,  4,  2,  1,  6,  2,  0,  1,  2],
       [ 1, 49,  0,  4,  0,  0,  2,  8,  2,  0,  7,  0,  0,  6,  2],
       [ 9, 68,  6, 29,  0,  0,  0,  8,  4,  1,  4,  0,  0,  0,  2],
       [ 5, 45,  1, 13,  1,  0,  5,  5,  0,  0,  6,  0,  2,  6,  2],
       [ 7, 57,  1, 10,  0,  0,  0,  9,  0,  1,  8,  0,  0,  6,  2],
       [ 5, 42,  2, 16,  0,  0,  0,  7,  2,  0,  5,  0,  0,  6,  2]],
      dtype=int64)

In [74]:
array_numeric.shape

(25000, 30)

In [75]:
df3['readmitted'].unique()


array(['<30', '<15', '>30', 'NO', '>60', 'UNKNOWN'], dtype=object)

In [76]:
# Patients who returned to hospital within 30 days, shall be counted as being RE-ADMITTED  
# Those with NO and >30 day tags , shall be counted as NOT RE-ADMITTED 

df3['readmitted'].replace(to_replace='<30',value=1,inplace=True)
df3['readmitted'].replace(to_replace='<15',value=1,inplace=True)
df3['readmitted'].replace(to_replace='NO',value=0,inplace=True)
df3['readmitted'].replace(to_replace='>30',value=0,inplace=True)
df3['readmitted'].replace(to_replace='>60',value=0,inplace=True)
df3['readmitted'].replace(to_replace='UNKNOWN',value=0,inplace=True)

In [77]:
df3['readmitted'].value_counts()

0    22168
1     2832
Name: readmitted, dtype: int64

In [78]:
df3.shape

(25000, 33)

In [79]:
y_train_test = pd.DataFrame(df3, columns = ['readmitted']).values
type(y_train_test)

numpy.ndarray

In [80]:
x_train_test = array_numeric
x_train_test[0:1,0:30]

array([[ 7, 62,  0, 11,  0,  0,  0,  7,  0,  1,  6,  1,  0,  3,  2,  2,
         1,  1,  1,  0,  1,  1,  3,  1,  1,  1,  2,  1,  0,  1]],
      dtype=int64)

In [81]:
obj_strata = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size=0.8, random_state=2323)
obj_strata.get_n_splits(x_train_test, y_train_test)

for train_index, test_index in obj_strata.split (x_train_test, y_train_test):
    print("TRAIN: ", train_index, "TEST: ", test_index)
    x_train, x_test = x_train_test[train_index], x_train_test[test_index]
    y_train, y_test = y_train_test[train_index], y_train_test[test_index]

TRAIN:  [ 5433   996  1218 ... 10458 12795 10573] TEST:  [16723 16569 20736 ... 11038 19790 10383]


In [82]:
print(sum(y_train))
print(sum(y_test))

[2266]
[566]


In [83]:
np.unique(y_train)
np.unique(y_test)

array([0, 1], dtype=int64)

In [84]:
x_train.shape
x_test.shape
y_train.shape
y_test.shape

(5000, 1)

In [85]:
def model_performance(model):
    RF_training_labels = model.predict(x_train)
    RF_test_labels = model.predict(x_test)
    Training_accuracy = model.score(x_train, y_train, sample_weight=None)
    Test_accuracy = model.score(x_test, y_test, sample_weight=None)
    F1_score_train = f1_score(y_train, RF_training_labels, average = 'weighted')
    F1_score_test = f1_score(y_test, RF_test_labels, average = 'weighted')
    Recall_train = recall_score(y_train, RF_training_labels, average = 'weighted') 
    Recall_test  = recall_score(y_test, RF_test_labels, average = 'weighted') 
    Precision_train = precision_score(y_train, RF_training_labels, average = 'weighted')
    Precision_test = precision_score(y_test, RF_test_labels, average = 'weighted')
    rf_cm_tr = confusion_matrix(y_train, RF_training_labels)
    rf_cm_te = confusion_matrix(y_test, RF_test_labels)
    print("Training_accuracy - ", Training_accuracy)
    print("Test_accuracy - ", Test_accuracy)
    print("F1_score_train - ", F1_score_train)
    print("F1_score_test - ", F1_score_test)
    print("Recall_train - ", Recall_train)
    print("Recall_test - ", Recall_test)
    print("Precision_train - ", Precision_train)
    print("Precision_test - ", Precision_test)
    print("Training_Confusion_Matrix - ", rf_cm_tr)
    print("Test Confusion Matrix - ", rf_cm_te)
    return RF_training_labels, RF_test_labels 

In [86]:
import xgboost
from xgboost import XGBClassifier
model =XGBClassifier(gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=11, seed=0, silent=True, subsample=1).fit(x_train,y_train)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[14:31:33] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [87]:
XG_training_labels, XG_test_labels = model_performance(model)

Training_accuracy -  0.4966
Test_accuracy -  0.4686
F1_score_train -  0.5759073567229896
F1_score_test -  0.5510967303691306
Recall_train -  0.4966
Recall_test -  0.4686
Precision_train -  0.8681597304265152
Precision_test -  0.8440322484677351
Training_Confusion_Matrix -  [[8017 9717]
 [ 351 1915]]
Test Confusion Matrix -  [[1914 2520]
 [ 137  429]]


In [89]:

print (" Test Confusion Matrix : ")
ls_test = [[1914,2520],[ 137 , 429]]
print (ls_test[0][0],"       ", ls_test[0][1])
print (ls_test[1][0],"       ", ls_test[1][1])

 Test Confusion Matrix : 
1914         2520
137         429
